# Steps

merge
- merge CHR with fips code using 'cbsacode' column
- merge SMART with CHR
- select columns
- aggregate the data

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
data_dir = "..\\data\\"

In [3]:
chr = pd.read_csv(f"{data_dir}combinedCHR_selectedColumns_calculated.csv")[1:]

In [4]:
smart_cbsa = pd.read_csv(f"{data_dir}20241025smart_with_cbsa.csv")

In [5]:
chr.head(3)

,%_65_and_older,%_american_indian_&_alaska_native,%_asian,%_below_18_years_of_age,%_female,%_hispanic,%_native_hawaiian/other_pacific_islander,%_not_proficient_in_english,%_rural,adult_obesity,adult_population,adult_population18_65,adult_smoking,children_eligible_for_free_or_reduced_price_lunch,children_in_poverty,children_in_single-parent_households,county,county_ranked,countycode,diabetes_prevalence,driving_alone_to_work,excessive_drinking,fipscode,high_school_graduation,hiv_prevalence,homicides,low_birthweight,mammography_screening,median_household_income,mental_health_providers,motor_vehicle_crash_deaths,physical_inactivity,poor_mental_health_days,poor_or_fair_health,poor_physical_health_days,population,population_below_18,premature_death,preventable_hospital_stays,primary_care_physicians,ratio_of_population_to_mental_health_providers.,ratio_of_population_to_primary_care_physicians.,sexually_transmitted_infections,some_college,state,statecode,teen_births,total_65_and_older,total_adult_obesity,total_adult_smoking,total_below_18_years_of_age,total_children_in_poverty,total_children_in_single-parent_households,total_excessive_drinking,total_female,total_homicides,total_mental_health_providers,total_motor_vehicle_crash_deaths,total_not_proficient_in_english,total_physical_inactivity,total_poor_mental_health_days,total_poor_or_fair_health,total_poor_physical_health_days,total_premature_death,total_primary_care_physicians,total_rural,total_sexually_transmitted_infections,total_uninsured_adults,total_violent_crime,unemployment,uninsured_adults,violent_crime,year
1,0.138,0.005,0.011,0.239,0.516,0.031,0.0,0.020,0.446,0.319,3583326.788,2933525.084,0.236,0.512,0.221,0.362,Alabama,NaN,0,0.119,0.835,0.126,1000.0,0.671,NaN,NaN,0.1021,0.625100,42586.0,0.000198,25.6,0.305,4.1,0.201,4.2,4708708.0,1125381.212,10189.2,87.9589,0.000798,5051.0,1253.675690,535.0,0.551,AL,1,53.2,649801.704,1.143081e+06,845665.121968,1125381.212,248709.247852,407387.998744,451499.175288,2429693.328,NaN,237813.535354,1205.429248,94174.160,1.092915e+06,643523.426667,720248.684388,659219.120000,479779.675536,59031.924223,2100083.768,25191.587800,501632.789364,20336.909852,0.101,0.171,431.9,2011.0
2,0.116,0.005,0.006,0.278,0.514,0.022,0.0,0.008,0.448,0.315,36645.832,30758.136,0.274,0.288,0.149,0.297,Autauga County,1.0,1,0.114,0.855,0.141,1001.0,0.740,169.9,NaN,0.0945,0.695122,51622.0,0.000020,28.3,0.327,4.1,0.258,5.2,50756.0,14110.168,9967.4,78.8536,0.000437,50354.0,2288.818182,307.8,0.554,AL,1,52.0,5887.696,1.154344e+04,10040.957968,14110.168,2102.415032,4190.719896,5167.062312,26088.584,NaN,25378.000000,14.363948,406.048,1.198319e+04,6936.653333,9454.624656,8797.706667,5059.053544,1161.712558,22738.688,156.226968,4767.511080,130.087628,0.089,0.155,256.3,2011.0
3,0.170,0.005,0.006,0.231,0.510,0.033,0.0,0.018,0.542,0.262,138326.182,107746.922,0.219,0.289,0.151,0.289,Baldwin County,1.0,3,0.098,0.829,0.194,1003.0,0.681,176.1,NaN,0.0882,0.724967,51957.0,0.000165,23.2,0.252,4.1,0.128,3.4,179878.0,41551.818,8321.8,63.9873,0.000817,6076.0,1223.694444,226.4,0.615,AL,1,49.6,30579.260,3.624146e+04,30293.433858,41551.818,6274.324518,12008.475402,26835.279308,91737.780,NaN,10901.696970,41.731696,3237.804,3.485820e+04,24583.326667,17705.751296,20386.173333,14969.087404,2201.157003,97493.876,407.243792,22519.106698,350.222466,0.088,0.209,194.7,2011.0


In [6]:
smart_cbsa.head(3)

,Year,Locationabbr,Locationdesc,Response,Sample_Size,Data_value,Confidence_limit_Low,Confidence_limit_High,Display_order,Data_value_unit,Data_value_type,Data_Value_Footnote_Symbol,Data_Value_Footnote,LocationID,QuestionID,RESPONSEID,GeoLocation,cbsacode,metropolitandivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis
0,2011,15804,"Camden, NJ Metropolitan Division",Yes,253,12.52,10.34,14.70,50,%,Crude Prevalence,NaN,NaN,15804,ADDEPEV2,RESP046,"(39.8098807, -74.8232086)",37980,15804.0,428.0,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Metropolitan Statistical Area
1,2020,15804,"Camden, NJ Metropolitan Division",Yes,338,18.15,15.82,20.48,2,%,Crude Prevalence,NaN,NaN,15804,ADDEPEV3,RESP046,"(39.8097724, -74.8232021)",37980,15804.0,428.0,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Metropolitan Statistical Area
2,2013,15804,"Camden, NJ Metropolitan Division",Yes,346,17.91,15.58,20.24,49,%,Crude Prevalence,NaN,NaN,15804,ADDEPEV2,RESP046,"(39.8098807, -74.8232086)",37980,15804.0,428.0,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Metropolitan Statistical Area


## Merge

In [7]:
print(smart_cbsa.shape, chr.shape)

(1735, 22) (38322, 73)


In [8]:
cbsa = pd.read_csv(f"{data_dir}cbsa2fipsxw.csv")
cbsa['metropolitandivisioncode'] = cbsa['metropolitandivisioncode'].fillna(cbsa['cbsacode']) # brutal force, alternatively. could dropna()
cbsa['fipscode'] = cbsa.fipsstatecode.astype(str) + cbsa.fipscountycode.astype(str).str.zfill(3)

cbsa = cbsa[["fipscode", "cbsacode"]]#, "metropolitandivisioncode","csacode", "cbsatitle", "metropolitanmicropolitanstatis", "fipscode"]]
cbsa.head(3)

,fipscode,cbsacode
0,1001,33860
1,1003,19300
2,1005,21640


In [9]:
print("chr.shape", chr.shape)
chr = chr.dropna(subset=['fipscode']) ## brutal
print("chr.shape", chr.shape)

print(cbsa.shape, chr.shape)

chr['fipscode'] = chr['fipscode'].astype(int)
cbsa['fipscode'] = cbsa['fipscode'].astype(int)

chr_cbsa = pd.merge(
    cbsa, chr, 
    on='fipscode', 
    how='inner'
)

chr_cbsa.shape


chr.shape (38322, 73)
chr.shape (38321, 73)
(1915, 2) (38321, 73)


(22020, 74)

In [10]:
# sanity check
chr['fipscode'] = chr['fipscode'].astype(int)
cbsa['fipscode'] = cbsa['fipscode'].astype(int)

merged_df = pd.merge(
    chr, cbsa, 
    on='fipscode', 
    how='left',
    indicator=True 
)

print("merged_df.shape:", merged_df.shape)

left_only_rows = merged_df[merged_df['_merge'] == 'left_only']

print("Left-only rows in 'chr':")
print(len(left_only_rows))


merged_df.shape: (38321, 75)
Left-only rows in 'chr':
16301


In [11]:
print(chr_cbsa.shape, smart_cbsa.shape)

(22020, 74) (1735, 22)


In [12]:
smart_cbsa.cbsacode

0       37980
1       37980
2       37980
3       37980
4       37980
        ...  
1730    45300
1731    45300
1732    41860
1733    41860
1734    41860
Name: cbsacode, Length: 1735, dtype: int64

In [13]:
print("Columns in smart_cbsa:", smart_cbsa.columns)
print("Columns in chr:", chr_cbsa.columns)


Columns in smart_cbsa: Index(['Year', 'Locationabbr', 'Locationdesc', 'Response', 'Sample_Size',
       'Data_value', 'Confidence_limit_Low', 'Confidence_limit_High',
       'Display_order', 'Data_value_unit', 'Data_value_type',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote', 'LocationID',
       'QuestionID', 'RESPONSEID', 'GeoLocation', 'cbsacode',
       'metropolitandivisioncode', 'csacode', 'cbsatitle',
       'metropolitanmicropolitanstatis'],
      dtype='object')
Columns in chr: Index(['fipscode', 'cbsacode', '%_65_and_older',
       '%_american_indian_&_alaska_native', '%_asian',
       '%_below_18_years_of_age', '%_female', '%_hispanic',
       '%_native_hawaiian/other_pacific_islander',
       '%_not_proficient_in_english', '%_rural', 'adult_obesity',
       'adult_population', 'adult_population18_65', 'adult_smoking',
       'children_eligible_for_free_or_reduced_price_lunch',
       'children_in_poverty', 'children_in_single-parent_households', 'county',
      

In [14]:
data = pd.merge(
    smart_cbsa, chr_cbsa, 
    left_on=['cbsacode', 'Year'], 
    right_on=['cbsacode', 'year'], 
    how='inner'
)
data.head(3)

,Year,Locationabbr,Locationdesc,Response,Sample_Size,Data_value,Confidence_limit_Low,Confidence_limit_High,Display_order,Data_value_unit,Data_value_type,Data_Value_Footnote_Symbol,Data_Value_Footnote,LocationID,QuestionID,RESPONSEID,GeoLocation,cbsacode,metropolitandivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,fipscode,%_65_and_older,%_american_indian_&_alaska_native,%_asian,%_below_18_years_of_age,%_female,%_hispanic,%_native_hawaiian/other_pacific_islander,%_not_proficient_in_english,%_rural,adult_obesity,adult_population,adult_population18_65,adult_smoking,children_eligible_for_free_or_reduced_price_lunch,children_in_poverty,children_in_single-parent_households,county,county_ranked,countycode,diabetes_prevalence,driving_alone_to_work,excessive_drinking,high_school_graduation,hiv_prevalence,homicides,low_birthweight,mammography_screening,median_household_income,mental_health_providers,motor_vehicle_crash_deaths,physical_inactivity,poor_mental_health_days,poor_or_fair_health,poor_physical_health_days,population,population_below_18,premature_death,preventable_hospital_stays,primary_care_physicians,ratio_of_population_to_mental_health_providers.,ratio_of_population_to_primary_care_physicians.,sexually_transmitted_infections,some_college,state,statecode,teen_births,total_65_and_older,total_adult_obesity,total_adult_smoking,total_below_18_years_of_age,total_children_in_poverty,total_children_in_single-parent_households,total_excessive_drinking,total_female,total_homicides,total_mental_health_providers,total_motor_vehicle_crash_deaths,total_not_proficient_in_english,total_physical_inactivity,total_poor_mental_health_days,total_poor_or_fair_health,total_poor_physical_health_days,total_premature_death,total_primary_care_physicians,total_rural,total_sexually_transmitted_infections,total_uninsured_adults,total_violent_crime,unemployment,uninsured_adults,violent_crime,year
0,2011,15804,"Camden, NJ Metropolitan Division",Yes,253,12.52,10.34,14.7,50,%,Crude Prevalence,NaN,NaN,15804,ADDEPEV2,RESP046,"(39.8098807, -74.8232086)",37980,15804.0,428.0,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Metropolitan Statistical Area,10003,0.120,0.003,0.040,0.238,0.514,0.080,0.001,0.047,0.056,0.272,407391.108,343235.028,0.192,0.317,0.117,0.331,New Castle County,1.0,3,0.079,0.788,0.208,0.696,495.3,NaN,0.0988,0.672444,63301.0,0.000542,11.5,0.220,3.4,0.116,3.2,534634.0,127242.892,7331.5,68.5172,0.000957,1844.0,1045.387795,425.2,0.632,DE,10,35.9,64156.080,110810.381376,78219.092736,127242.892,14887.418364,42117.397252,84737.350464,274801.876,NaN,9864.095941,61.482910,25127.798,89626.043760,60591.853333,47257.368528,57027.626667,39196.691710,5588.998673,29939.504,2273.263768,41531.438388,4100.642780,0.081,0.121,767.0,2011.0
1,2011,15804,"Camden, NJ Metropolitan Division",Yes,253,12.52,10.34,14.7,50,%,Crude Prevalence,NaN,NaN,15804,ADDEPEV2,RESP046,"(39.8098807, -74.8232086)",37980,15804.0,428.0,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Metropolitan Statistical Area,24015,0.119,0.003,0.011,0.247,0.505,0.026,0.000,0.016,0.524,0.294,75899.388,63904.664,0.252,0.245,0.108,0.264,Cecil County,1.0,15,0.092,0.810,0.167,0.723,149.6,NaN,0.0770,0.576779,66131.0,0.000150,23.6,0.261,4.1,0.143,4.1,100796.0,24896.612,8538.2,111.0410,0.000370,6663.0,2701.324324,183.1,0.531,MD,24,40.6,11994.724,22314.420072,19126.645776,24896.612,2688.834096,6572.705568,12675.197796,50901.980,NaN,6719.733333,23.787856,1612.736,19809.740268,13775.453333,10853.612484,13775.453333,8606.164072,2722.826839,52817.104,184.557476,8882.748296,539.964172,0.090,0.139,535.7,2011.0
2,2011,15804,"Camden, NJ Metropolitan Division",Yes,253,12.52,10.34,14.7,50,%,Crude Prevalence,NaN,NaN,15804,ADDEPEV2,RESP046,"(39.8098807, -74.8232086)",37980,15804.0,428.0,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Metropolitan Statistical Area,34005,0.136,0.003,0.041,0.229,0.504,0.058,0.000,0.042,0.074,0.262,343949.268,283278.580,0.175,0.130,0.068,0.235,Burlington County,1.0,5,0.090,0.818,0.155,0.837

In [15]:
data.columns

Index(['Year', 'Locationabbr', 'Locationdesc', 'Response', 'Sample_Size',
       'Data_value', 'Confidence_limit_Low', 'Confidence_limit_High',
       'Display_order', 'Data_value_unit', 'Data_value_type',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote', 'LocationID',
       'QuestionID', 'RESPONSEID', 'GeoLocation', 'cbsacode',
       'metropolitandivisioncode', 'csacode', 'cbsatitle',
       'metropolitanmicropolitanstatis', 'fipscode', '%_65_and_older',
       '%_american_indian_&_alaska_native', '%_asian',
       '%_below_18_years_of_age', '%_female', '%_hispanic',
       '%_native_hawaiian/other_pacific_islander',
       '%_not_proficient_in_english', '%_rural', 'adult_obesity',
       'adult_population', 'adult_population18_65', 'adult_smoking',
       'children_eligible_for_free_or_reduced_price_lunch',
       'children_in_poverty', 'children_in_single-parent_households', 'county',
       'county_ranked', 'countycode', 'diabetes_prevalence',
       'driving_alone_to_w

In [16]:

columns_to_keep = [
    'Locationabbr', 'year','Locationdesc',
    'Sample_Size', 'Data_value', 'Data_value_unit', 'Data_value_type',
     'fipscode',
    'adult_population', 'adult_population18_65', 'population', 'total_65_and_older',
    'total_adult_obesity', 'total_adult_smoking', 'total_below_18_years_of_age',
    'total_children_in_poverty', 'total_children_in_single-parent_households',
    'total_excessive_drinking', 'total_female', 'total_homicides',
    'total_mental_health_providers', 'total_motor_vehicle_crash_deaths',
    'total_not_proficient_in_english', 'total_physical_inactivity',
    'total_poor_mental_health_days', 'total_poor_or_fair_health',
    'total_poor_physical_health_days', 'total_premature_death',
    'total_primary_care_physicians', 'total_rural',
    'total_sexually_transmitted_infections', 'total_uninsured_adults',
    'total_violent_crime',"cbsacode", "metropolitandivisioncode","csacode", "cbsatitle", "metropolitanmicropolitanstatis"
]

selected_df = data[columns_to_keep]
selected_df.to_csv(f"{data_dir}20241025selected_smart_chr_cbsa.csv", index=False)


In [17]:
sum_cols = [
    'adult_population', 'adult_population18_65', 'population', 'total_65_and_older',
    'total_adult_obesity', 'total_adult_smoking', 'total_below_18_years_of_age',
    'total_children_in_poverty', 'total_children_in_single-parent_households',
    'total_excessive_drinking', 'total_female', 'total_homicides',
    'total_mental_health_providers', 'total_motor_vehicle_crash_deaths',
    'total_not_proficient_in_english', 'total_physical_inactivity',
    'total_poor_mental_health_days', 'total_poor_or_fair_health',
    'total_poor_physical_health_days', 'total_premature_death',
    'total_primary_care_physicians', 'total_rural',
    'total_sexually_transmitted_infections', 'total_uninsured_adults',
    'total_violent_crime'
]

aggregated_df = selected_df.groupby([
    'year', 'Locationdesc', 'Sample_Size', 'Data_value', #'Locationabbr'
    'Data_value_unit', 'Data_value_type', "cbsacode"
]).agg(
    **{col: (col, 'sum') for col in sum_cols} # using ** unpacking dict
).reset_index()

print(aggregated_df.shape)


(1605, 32)


In [18]:
aggregated_df.groupby("year").size()

year
2011.0    179
2012.0    172
2013.0    136
2014.0    122
2015.0    123
2016.0    134
2017.0    129
2018.0    127
2019.0    129
2020.0    112
2021.0    119
2022.0    123
dtype: int64

## Save into csv

In [19]:
aggregated_df.sort_values(by=['Locationdesc', 'year'], ascending=[True, True]).to_csv(f"{data_dir}20241025agg_smart_chr.csv", index=False)

In [20]:
aggregated_df.sort_values(by=['Locationdesc', 'year'], ascending=[True, True])

,year,Locationdesc,Sample_Size,Data_value,Data_value_unit,Data_value_type,cbsacode,adult_population,adult_population18_65,population,total_65_and_older,total_adult_obesity,total_adult_smoking,total_below_18_years_of_age,total_children_in_poverty,total_children_in_single-parent_households,total_excessive_drinking,total_female,total_homicides,total_mental_health_providers,total_motor_vehicle_crash_deaths,total_not_proficient_in_english,total_physical_inactivity,total_poor_mental_health_days,total_poor_or_fair_health,total_poor_physical_health_days,total_premature_death,total_primary_care_physicians,total_rural,total_sexually_transmitted_infections,total_uninsured_adults,total_violent_crime
0,2011.0,"Aberdeen, SD Micropolitan Statistical Area",88,16.75,%,Crude Prevalence,10100,30312.839000,23626.885000,39139.0,6685.954000,8924.320295,5720.877944,8826.161000,1145.953026,2023.366088,6192.435272,20078.096000,0.704080,inf,5.107186,580.270000,8501.832617,3026.186667,3780.488759,3887.666667,2055.415364,529.733192,14460.996000,79.103388,3323.081065,0.000000
487,2014.0,"Aberdeen, SD Micropolitan Statistical Area",105,19.36,%,Crude Prevalence,10100,31817.000001,25031.000000,41357.0,6786.000000,10033.372000,5432.763000,9539.999999,1193.423000,2255.656177,6567.681000,21078.992158,0.000000,0.000000,4.168915,236.587419,8718.757000,3258.536667,3503.131000,3846.566667,2269.765419,429.563140,14811.124251,133.732192,3382.513897,68.681027
609,2015.0,"Aberdeen, SD Micropolitan Statistical Area",84,14.29,%,Crude Prevalence,10100,32122.000001,25260.000000,41948.0,6862.000001,9547.014000,5485.478000,9825.999999,1239.094000,2401.510485,6630.726000,21341.000001,0.000000,129.454111,4.027708,360.664074,8141.064000,3307.086667,3537.146000,3901.676667,2239.679700,510.165404,14992.533711,116.714265,3319.492686,76.653985
866,2017.0,"Aberdeen, SD Micropolitan Statistical Area",73,16.05,%,Crude Prevalence,10100,32650.000002,25558.000000,42784.0,7092.000002,10418.099001,4656.683710,10133.999998,1298.228000,2742.892285,6417.598807,21645.000002,0.000000,118.446946,4.589678,599.821412,7132.246000,3490.526396,3167.855440,4003.362577,2354.264899,711.004036,15204.192576,109.490055,2884.542145,87.999550
995,2018.0,"Aberdeen, SD Micropolitan Statistical Area",89,20.73,%,Crude Prevalence,10100,32837.999999,25643.000000,43080.0,7194.999999,10723.698000,4895.434349,10242.000001,1247.380000,2662.586027,6061.783224,21763.999999,0.000000,116.870258,4.283354,648.397410,7385.762000,4038.901231,3447.234083,4204.171943,2549.548269,790.365300,15256.287099,138.552248,3058.224056,88.777785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,2018.0,"Youngstown-Warren-Boardman, OH-PA Metropolitan...",118,21.67,%,Crude Prevalence,49660,343279.000000,256204.000005,431833.0,87074.999995,107255.282000,75392.518022,88554.000000,24924.388000,37860.566577,56180.279481,220999.000002,33.614978,2669.306955,48.436734,3466.834038,93009.063000,58571.077290,58267.648463,55897.048934,37397.298733,6644.177518,89910.093602,1914.453132,23312.580343,1204.830570
1250,2019.0,"Youngstown-Warren-Boardman, OH-PA Metropolitan...",118,21.98,%,Crude Prevalence,49660,342779.000006,253607.999996,430176.0,89171.000010,112775.109001,75284.482986,87396.999994,23279.268998,37436.731085,56091.959076,220231.000000,36.405060,2278.542956,44.892828,2903.458276,92966.065002,58340.519733,58188.590883,55684.091960,41997.847294,6504.596301,89483.858539,2017.227068,19019.286122,1116.462305
1362,2020.0,"Youngstown-Warren-Boardman, OH-PA Metropolitan...",142,22.97,%,Crude Prevalence,49660,341488.999999,250413.000001,428269.0,91075.999998,118408.303999,67583.990450,86780.000001,21658.572000,36925.029833,52023.312736,219095.000003,36.459120,1943.215059,42.141212,3089.630406,101097.636000,66164.975470,60703.440741,58483.565807,41881.744199,6552.798699,88982.413679,2113.596676,20784.044310,1111.872348
1481,2021.0,"Youngstown-Warren-Boardman, OH-PA M